## Baseline Scenario

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt

# predefined smart city scenario
from mobile_env.scenarios.smart_city import MComSmartCity

In [ ]:
env = gym.make("mobile-smart_city-smart_city_handler-v0", render_mode="rgb_array")

print(f"\nSmart city environment with {env.NUM_USERS} users, {env.NUM_SENSORS} sensors and {env.NUM_STATIONS} cells.")

In [ ]:
from IPython import display
import pandas as pd
import matplotlib.pyplot as plt
import time

# Reset environment and initialize variables
done = False
obs, info = env.reset()
all_performance_means = []  # List to store mean metrics for each episode
episode_infos = []

# Run for 100 time steps for 100 episodes
for episode in range(100):
    obs, info = env.reset()
    print(f"Starting Episode {episode + 1}")
    episode_start_time = time.time()  # Start timing the episode

    for t in range(100):
        # Perform dummy action
        dummy_action = env.action_space.sample()
        #dummy_action = (0.50, 0.50)
        obs, reward, terminated, truncated, info = env.step(dummy_action)

        # Check termination condition
        if terminated or truncated:
            break
    
    # Calculate and store the episode duration
    episode_duration = time.time() - episode_start_time
    print(f"Completed Episode {episode + 1}: {episode_duration}")

    # Load results after the episode
    _, performance_results, _, _, _, _ = env.monitor.load_results()

    # Compute mean of metrics over the episode
    performance_mean = performance_results.mean()  # Calculate mean for each metric
    all_performance_means.append(performance_mean)

    # Add episode-level information to the list
    episode_info = {
        "Episode": episode + 1,
        "Total Reward": info.get("episode reward"),
        "Queue Size Transferred Jobs UE": info.get("bs trans. ue"),
        "Queue Size Transferred Jobs Sensor": info.get("bs trans. ss"),
        "Queue Size Accomplished Jobs UE": info.get("bs accomp. us"),
        "Queue Size Accomplished Jobs Sensor": info.get("bs accomp. ss"),
    }
    episode_infos.append(episode_info)

    print(f"Mean Reward for Episode: {episode_info['Total Reward']/100:.2f}")


# Convert all_kpi_means and episode_infos to DataFrames
episode_infos_df = pd.DataFrame(episode_infos)
all_performance_means_df = pd.DataFrame(all_performance_means)

# Display mean KPI metrics for each episode
print(episode_infos_df)
print(all_performance_means_df)

In [ ]:
# Plot each metric in a separate plot
required_metrics = ["reward", "total throughput ue", "total throughput sensor", "total delayed packets", "total aori", "total aosi"]

for metric in required_metrics:
    plt.figure(figsize=(10, 6))
    plt.plot(all_performance_means_df.index, all_performance_means_df[metric], marker='o', label=metric)
    plt.title(f"Mean {metric} Over Episodes", fontsize=16)
    plt.xlabel("Episode", fontsize=14)
    plt.ylabel("Value", fontsize=14)
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
import numpy as np

# Function to calculate and plot the CDF for each metric
def plot_cdf_metrics(metrics_df, metrics_list):
    for metric in metrics_list:
        plt.figure(figsize=(10, 6))
        
        # Get the values for the metric
        metric_values = metrics_df[metric].dropna().values  # Drop NaN values
        
        # Sort the values and compute the CDF
        sorted_values = np.sort(metric_values)
        cdf = np.arange(1, len(sorted_values) + 1) / len(sorted_values)
        
        # Plot the CDF
        plt.plot(sorted_values, cdf, marker='o', label=f"CDF of {metric}")
        plt.title(f"CDF of {metric}", fontsize=16)
        plt.xlabel(f"{metric} Value", fontsize=14)
        plt.ylabel("Cumulative Probability", fontsize=14)
        plt.grid(True)
        plt.legend()
        plt.show()

# Metrics to plot
required_metrics = ["reward", "total throughput ue", "total throughput sensor", "total delayed packets", "total aori", "total aosi"]

# Call the CDF plot function
plot_cdf_metrics(all_performance_means_df, required_metrics)
